In [3]:
import pymc as pm
import numpy as np
import arviz as az

%load_ext lab_black
%load_ext watermark

# Lister

Adapted from [Unit 10: lister.odc](https://raw.githubusercontent.com/areding/6420-pymc/main/original_examples/Codes4Unit10/lister.odc).

Data can be found [here](https://raw.githubusercontent.com/areding/6420-pymc/main/data/r.txt).

Associated lecture video: Unit 10 lesson 1

## Problem statement

Sir Joseph Lister (1827--1912), Professor of Surgery at Glasgow University, influenced by Pasteur's ideas, found that a wound wrapped in bandages treated by Carbolic acid (phenol) would often not become infected.

Here are Lister's data on treating open fractures and amputations:

Period    Carbolic acid used    Results

1864--1866     No       Treated   34 patients, 15 died and 19 recovered

1867--1870     Yes     Treated   40 patients,   6 died and 34 recovered


Estimate and interpret the risk difference,  risk ratio, and   odds ratio


notes:
- Wishart!
- instead use LKJ distribution
- possibly useful paper:
    - https://arxiv.org/pdf/1809.04746.pdf


Unfinished! This is not working correctly right now.

In [4]:
n1 = 34
y1 = 15
n2 = 40
y2 = 6
mu0 = np.array([-0.5, -1.5])
S = np.array([[0.1, 0], [0, 0.1]])

In [11]:
with pm.Model() as m:
    sd_dist = pm.Normal.dist(0, 1, shape=2)
    chol, corr, stds = pm.LKJCholeskyCov(
        "chol", n=2, eta=1, sd_dist=sd_dist, compute_corr=True
    )

    mu = pm.MvNormal("mu", mu0, chol=chol, shape=2)

    p1 = mu[0]
    p2 = mu[1]

    pm.Binomial("y1", logit_p=p1, n=n1, observed=y1)
    pm.Binomial("y2", logit_p=p2, n=n2, observed=y2)

    pm.Deterministic("rd", p1 - p2)
    pm.Deterministic("rr", p1 / p2)
    pm.Deterministic("or", (p1 / (1 - p1)) / (p2 / (1 - p2)))

    trace = pm.sample(3000)

Auto-assigning NUTS sampler...
INFO:pymc:Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...
INFO:pymc:Initializing NUTS using jitter+adapt_diag...
Multiprocess sampling (4 chains in 4 jobs)
INFO:pymc:Multiprocess sampling (4 chains in 4 jobs)
NUTS: [chol, mu]
INFO:pymc:NUTS: [chol, mu]


/Users/aaron/mambaforge/envs/pymc_env/lib/python3.10/site-packages/aesara/compile/function/types.py:975: RuntimeWarning: invalid value encountered in accumulate
  self.vm()
/Users/aaron/mambaforge/envs/pymc_env/lib/python3.10/site-packages/aesara/compile/function/types.py:975: RuntimeWarning: invalid value encountered in accumulate
  self.vm()
/Users/aaron/mambaforge/envs/pymc_env/lib/python3.10/site-packages/aesara/compile/function/types.py:975: RuntimeWarning: invalid value encountered in accumulate
  self.vm()
/Users/aaron/mambaforge/envs/pymc_env/lib/python3.10/site-packages/aesara/compile/function/types.py:975: RuntimeWarning: invalid value encountered in accumulate
  self.vm()
Sampling 4 chains for 1_000 tune and 3_000 draw iterations (4_000 + 12_000 draws total) took 33 seconds.
INFO:pymc:Sampling 4 chains for 1_000 tune and 3_000 draw iterations (4_000 + 12_000 draws total) took 33 seconds.
There were 390 divergences after tuning. Increase `target_accept` or reparameterize.
ERR

In [13]:
az.summary(trace)

/Users/aaron/mambaforge/envs/pymc_env/lib/python3.10/site-packages/arviz/stats/diagnostics.py:586: RuntimeWarning: invalid value encountered in double_scalars
  (between_chain_variance / within_chain_variance + num_samples - 1) / (num_samples)


,mean,sd,hdi_3%,hdi_97%,mcse_mean,mcse_sd,ess_bulk,ess_tail,r_hat
mu[0],-0.313,0.319,-0.870,0.304,0.011,0.008,805.0,1342.0,1.01
mu[1],-1.737,0.414,-2.489,-0.964,0.015,0.011,709.0,3397.0,1.02
chol[0],3.486,4.049,0.032,11.328,0.100,0.071,465.0,312.0,1.01
chol[1],-0.015,3.817,-7.870,7.987,0.061,0.054,4495.0,3169.0,1.03
chol[2],2.780,3.391,0.083,9.223,0.102,0.072,89.0,18.0,1.04
"chol_corr[0, 0]",1.000,0.000,1.000,1.000,0.000,0.000,12000.0,12000.0,NaN
"chol_corr[0, 1]",-0.038,0.618,-0.999,0.913,0.016,0.012,2431.0,2367.0,1.00
"chol_corr[1, 0]",-0.038,0.618,-0.999,0.913,0.016,0.012,2431.0,2367.0,1.00
"chol_corr[1, 1]",1.000,0.000,1.000,1.000,0.000,0.000,459.0,301.0,1.01
chol_stds[0],3.486,4.049,0.032,11.328,0.100,0.071,465.0,312.0,1.01


In [14]:
%watermark -n -u -v -iv -p aesara,aeppl

Last updated: Mon Jul 25 2022

Python implementation: CPython
Python version       : 3.10.5
IPython version      : 8.4.0

aesara: 2.7.3
aeppl : 0.0.31

arviz: 0.12.1
pymc : 4.0.1
numpy: 1.23.0

